# Notebook med API'er som benyttes i beregningene
- Alle spørringer lagres i egen datamappe

## Laster inn nødvendige pakker

In [ ]:
import pandas as pd
import requests
import io

## Angi årgang som skal hentes
- Årgangen som angis vil gjelde for hele arket
- Skal det hentes flere årganger må arket kjøres flere ganger

Mappen der filene skal lagres må allerede eksistere

In [ ]:
aar=input('Angi årgang for filene:' )
aar_1t = str(int(aar) - 1)
aar_t1 = str(int(aar) + 1)
print("Du lager nå filer for " + aar)

### Art

In [ ]:
#API mot KOSTRA regnskapsarter. 
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/259/codes?from=" + aar + "-01-01&to=" + aar_t1 + "-01-01&csvSeparator=;", headers=headers).content
art = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
art = art[['code', 'validFromInRequestedRange']]
art.rename(columns={'code':'art', 'validFromInRequestedRange':'periode'}, inplace = True)
art['periode']=pd.DatetimeIndex(art['periode']).year
art = art[~art.art.str.contains("AG")]
art['art'] = art['art'].replace('I','', regex=True).replace('A','', regex=True).replace('D','', regex=True)
art.drop_duplicates().astype(str).info()
art.head(2)

In [ ]:
#art.to_parquet("../../../Testdata/arbeidsdata/klass/" + aar + "/art.parquet")
art.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/art.parquet")

### Funksjon

In [ ]:
#API mot KOSTRA regnskapsfunksjoner
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/277/codes?from=" + aar + "-01-01&to=" + aar_t1 + "-01-01&csvSeparator=;", headers=headers).content
funksjon = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
funksjon = funksjon[['code', 'validFromInRequestedRange']]
funksjon.rename(columns={'code':'funksjon', 'validFromInRequestedRange':'periode'}, inplace = True)
funksjon['periode']=pd.DatetimeIndex(funksjon['periode']).year
funksjon = funksjon[~funksjon.funksjon.str.contains("F")]
funksjon.drop_duplicates().astype(str).info()
funksjon.head(2)

In [ ]:
funksjon.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/funksjon.parquet")

### Kommuner

In [ ]:
#API mot KOSTRA-kommuner med tilhørende regionsgrupperinger
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/231/codes?from=" + aar + "-01-01&to=" + aar_t1 + "-01-01&csvSeparator=;", headers=headers).content
komm = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
komm = komm[['code', 'validFromInRequestedRange']]
komm.rename(columns={'code':'region', 'validFromInRequestedRange':'periode'}, inplace = True)
komm['periode']=pd.DatetimeIndex(komm['periode']).year
komm = komm[~komm.region.str.contains("E")]
komm.drop_duplicates().astype(str).info()
komm.head(2)

In [ ]:
komm.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/komm.parquet")

### Fylkeskommuner

In [ ]:
#API mot KOSTRA-fylkeskommuner med tilhørende regionsgrupperinger
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/232/codes?from=" + aar + "-01-01&to=" + aar_t1 + "-01-01&csvSeparator=;", headers=headers).content
fykomm = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
fykomm = fykomm[['code', 'validFromInRequestedRange']]
fykomm.rename(columns={'code':'region', 'validFromInRequestedRange':'periode'}, inplace = True)
fykomm['periode']=pd.DatetimeIndex(fykomm['periode']).year
fykomm = fykomm[~fykomm.region.str.contains("E")]
fykomm.drop_duplicates().astype(str).info()
fykomm.head(2)

In [ ]:
fykomm.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fykomm.parquet")

### Kommunehierarki

In [ ]:
kommuner = komm[(komm['region']!= '2111')].copy().astype(str)

In [ ]:
EAKUO = kommuner[(kommuner['region']!= '0301')].copy()
EAKUO['to'] = 'EAKUO'

In [ ]:
EAK = kommuner.copy()
EAK['to'] = 'EAK'

In [ ]:
EKA = kommuner.copy()
EKA['to'] = 'EKA' + EKA['region'].str[0:2]

In [ ]:
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/112/correspondsAt?targetClassificationId=131&date=" + aar + "-01-01", headers=headers).content
EKG = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ',')
EKG = EKG[['sourceCode', 'targetCode']]
EKG.rename(columns={'sourceCode':'to', 'targetCode':'region'}, inplace = True)
EKG = EKG.astype({'region':str})
EKG['region']=EKG['region'].apply(lambda x: '{0:0>4}'.format(x))
EKG.to_parquet("../../../Testdata/arbeidsdata/KostraRegnskap/kommunegrupper.parquet")
EKG['periode'] = aar

In [ ]:
kommunehierarki = pd.concat([EAKUO, EAK, EKA, EKG]).drop_duplicates()

In [ ]:
kommunehierarki['sign'] = '+'
kommunehierarki.rename(columns={'region':'from'}, inplace = True)

In [ ]:
kommunehierarki.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/kommunehierarki.parquet")

### Fylkeskommunehierarki

In [ ]:
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/152/correspondsAt?targetClassificationId=127&date=" + aar + "-01-01", headers=headers).content
EAFKL = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ',')
EAFKL = EAFKL[['sourceCode', 'targetCode']]
EAFKL.rename(columns={'sourceCode':'to', 'targetCode':'region'}, inplace = True)
EAFKL = EKG.astype({'region':str})
EAFKL['region']=EKG['region'].apply(lambda x: '{0:0>4}'.format(x))
EAFKL.to_parquet("../../../Testdata/arbeidsdata/KostraRegnskap/kommunegrupper.parquet")
EAFKL['periode'] = aar

In [ ]:
EAFKUO = fykomm[(fykomm['region']!= '0301')].copy().astype(str)
EAFKUO['to'] = 'EAFKUO'

In [ ]:
EAFK = fykomm.copy().astype(str)
EAFK['to'] = 'EAK'

In [ ]:
fylkeskommunehierarki = pd.concat([EAFKUO, EAFK, EAFKL]).drop_duplicates()

In [ ]:
fylkeskommunehierarki['sign'] = '+'
fylkeskommunehierarki.rename(columns={'region':'from'}, inplace = True)

In [ ]:
fylkeskommunehierarki.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/fylkeskommunehierarki.parquet")

### Bydeler

In [ ]:
#API mot KOSTRA-fylkeskommuner med tilhørende regionsgrupperinger
headers = {'content-type': 'text/csv', 'charset' : 'ISO-8859-1'}
response = requests.get("http://data.ssb.no/api/klass/v1/classifications/241/codes?from=" + aar + "-01-01&to=" + aar_t1 + "-01-01&csvSeparator=;", headers=headers).content
bydel = pd.read_csv(io.StringIO(response.decode('ISO-8859-1')), sep = ';')

#Ordner datasettet til ønsket struktur
bydel = bydel[['code', 'validFromInRequestedRange']]
bydel.rename(columns={'code':'region', 'validFromInRequestedRange':'periode'}, inplace = True)
bydel['periode']=pd.DatetimeIndex(bydel['periode']).year
bydel = bydel[~bydel.region.str.contains("E")]
bydel.drop_duplicates().astype(str).info()
bydel.head(2)

In [ ]:
bydel = bydel[(bydel['region']!= '030199') & (bydel['region']!= 'EAB')].copy().astype(str)
bydel['to'] = 'EAB'
bydel['sign'] = '+'
bydel.rename(columns={'region':'from'}, inplace = True)

In [ ]:
bydel.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/bydelshierarki.parquet")

### Regionshierarki

In [ ]:
regionshierarki = pd.concat([kommunehierarki, fylkeskommunehierarki, bydel])

In [ ]:
regionshierarki.to_parquet("/ssb/stamme03/komakro/pilot_python/arbeidsdata/klass/" + aar + "/regionshierarki.parquet")